In [1]:
#Based on Pycon2015 presentation "Losing your Loops Fast Numerical Computing with NUmpy" - Jake Vander Plas

In [2]:
import numpy as np

## Comparison with Fortran

In [3]:
def func_python(N):
        d=0.0
        for i in range(N):
            d+= (i%3-1)*i
        return d

In [4]:
%timeit func_python(10000)

832 µs ± 48.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%load_ext fortranmagic

/home/leo/.local/lib/python3.6/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [6]:
%%fortran
subroutine func_fort(n,d)
    integer, intent(in) :: n
    double precision, intent(out) :: d
    integer :: i
    d = 0
    do i = 0, n-1
        d = d+ (mod(i,3)-1)*i
    end do
end subroutine func_fort

In [7]:
%timeit func_fort(10000)

12 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Ufuncs vs Lists

In [8]:
a = list(range(100000))

In [9]:
%timeit [val + 5 for val in a]

4.64 ms ± 97 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
a = np.array(a)

In [11]:
%timeit a+5

46.2 µs ± 895 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


~92x faster

## Numpy Aggregations

In [12]:
from random import random

In [13]:
c=[random() for i in range (100000)]

In [14]:
%timeit min(c)

1.08 ms ± 64.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
c = np.array(c)

In [16]:
%timeit c.min()

27 µs ± 945 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Nearest Neighbors example

trying to find $min(D_{i,j})$ where $D^2_{i,j}=(x_i-x_j)^2+(y_i-y_j)^2$

In [17]:
#3D cloud of points
n=1000
x=np.random.random((n,3))

In [18]:
#Broadcasting to find pairwise differences
def NNeighbors(x,n):
    diff = x.reshape(n,1,3) - x
    #Aggregate to find pairwise distances
    D = (diff ** 2).sum(2)
    #Ignore self neighbors by setting their values to infinity (diagonal)
    i = np.arange(n)
    D[i,i]=np.inf
    i = np.argmin(D,1)
    return i
i=NNeighbors(x,n)
print(i[:10])

[290 811  71 526 384 214 497 437 689  11]


In [19]:
# checking it with scikit-learn
from sklearn.neighbors import NearestNeighbors
d, i = NearestNeighbors().fit(x).kneighbors(x,2)
print(i[:10,1])


[290 811  71 526 384 214 497 437 689  11]


In [20]:
%timeit NNeighbors(x,n)

38.2 ms ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit NearestNeighbors(algorithm='ball_tree').fit(x).kneighbors(x,2)

2.24 ms ± 41 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit NearestNeighbors(algorithm='brute').fit(x).kneighbors(x,2)

12.2 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## OpenMP and Fortran

In [23]:
%%fortran --f90flags='-fopenmp' --extra='-lgomp'
#%%fortran -vvv --f90flags='-fopenmp' --extra='-lgomp'
subroutine my_parallel_function()
    use omp_lib
    implicit none
    integer ( kind = 4 ) id
    real ( kind = 8 ) wtime
    wtime = omp_get_wtime ( )
    write ( *, '(a,i8)' ) &
        '  The number of processors available = ', omp_get_num_procs ( )
    write ( *, '(a,i8)' ) &
        '  The number of threads available    = ', omp_get_max_threads ( )
end subroutine

In [24]:
my_parallel_function()

## Using Cython
https://people.duke.edu/~ccc14/sta-663/FromCompiledToPython.html

### Creating files
the following cells create the files, fib.pyx

Function to be imported

In [25]:
%%file fib.h

double fib(int n);

Overwriting fib.h


In [26]:
%%file fib.c

double fib(int n) {
    double a = 0, b = 1;
    for (int i=0; i<n; i++) {
        double tmp = b;
        b = a;
        a += tmp;
     }
    return a;
}

Overwriting fib.c


In [27]:
%%file fib2.pyx

cimport fib

def fib(n):
    return fib.fib(n)

Overwriting fib2.pyx


In [28]:
%%file setup.py
from distutils.core import setup, Extension
from Cython.Build import cythonize

ext = Extension("fib2",
              sources=["fib2.pyx", "fib.c"])

setup(name = "cython_fib",
      ext_modules = cythonize(ext))

Overwriting setup.py


In [29]:
! python setup.py build_ext -i

Compiling fib2.pyx because it changed.
[1/1] Cythonizing fib2.pyx
running build_ext
building 'fib2' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fno-strict-aliasing -Wdate-time -D_FORTIFY_SOURCE=2 -g -fdebug-prefix-map=/build/python2.7-3hk45v/python2.7-2.7.15~rc1=. -fstack-protector-strong -Wformat -Werror=format-security -fPIC -I. -I/usr/include/python2.7 -c fib2.c -o build/temp.linux-x86_64-2.7/fib2.o
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fno-strict-aliasing -Wdate-time -D_FORTIFY_SOURCE=2 -g -fdebug-prefix-map=/build/python2.7-3hk45v/python2.7-2.7.15~rc1=. -fstack-protector-strong -Wformat -Werror=format-security -fPIC -I. -I/usr/include/python2.7 -c fib.c -o build/temp.linux-x86_64-2.7/fib.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -fno-strict-aliasing -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -Wdate-time -D_FORTIFY_SOU

In [ ]:
import fib2

fib2.fib(100)